In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import random as rand

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
import os
os.chdir('../../')

In [3]:
from programs import lstm_tool

In [5]:
df = pd.read_excel('./documents/nov_nine_var.xlsx').iloc[:,1::].to_numpy()

scaler = MinMaxScaler()
scale_df = scaler.fit_transform(df[:,0:21])

In [ ]:
def noiser(data):
    data[rand.randint(0, len(data))] = data[rand.randint(0, len(data))]


In [6]:
sequence, state = lstm_tool.make_sequence(scale_df, 120)

label = np.concatenate((np.ones(state.shape[0]), np.ones(state.shape[0]), np.zeros(state.shape[0]), np.zeros(state.shape[0]), np.zeros(state.shape[0])))

sequence = np.concatenate((sequence, sequence, sequence, sequence, sequence))

state = np.concatenate((state, state, np.random.permutation(state), np.random.permutation(state), np.random.permutation(state)))

In [7]:
sequence.shape

(1550, 120, 21)

In [8]:
sequence_train, sequence_test, state_train, state_test, label_train, label_test = train_test_split(sequence, state, label, test_size=0.3, shuffle=True)

In [9]:
lstm_input = tf.keras.layers.Input(shape=(120, 21))

input_lstm_layer = tf.keras.layers.LSTM(256, activation='tanh', kernel_initializer=tf.keras.initializers.GlorotUniform(), return_sequences=True)(lstm_input)
hidden_lstm_layer0 = tf.keras.layers.LSTM(256, activation='tanh', kernel_initializer=tf.keras.initializers.GlorotUniform(), return_sequences=False)(input_lstm_layer)
dropout_lstm_layer = tf.keras.layers.Dropout(0.2)(hidden_lstm_layer0)
hidden_lstm_layer1 = tf.keras.layers.Dense(128, activation='relu')(dropout_lstm_layer)
hidden_lstm_layer2 = tf.keras.layers.Dense(128, activation='relu')(hidden_lstm_layer1)

lstm_layer = tf.keras.models.Model(lstm_input, hidden_lstm_layer2)

state_input = tf.keras.layers.Input(shape=(21,))

input_state_layer = tf.keras.layers.Dense(64, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(state_input)
dropout_state_layer = tf.keras.layers.Dropout(0.3)(input_state_layer)
hidden_state_layer0 = tf.keras.layers.Dense(64, activation='relu')(dropout_state_layer)
hidden_state_layer1 = tf.keras.layers.Dense(64, activation='relu')(hidden_state_layer0)

state_layer = tf.keras.models.Model(state_input, hidden_state_layer1)

input_concat_layer = tf.keras.layers.concatenate([lstm_layer.output, state_layer.output])
hidden_concat_layer0 = tf.keras.layers.Dense(32, activation='relu')(input_concat_layer)
dropout_concat_layer = tf.keras.layers.Dropout(0.3)(hidden_concat_layer0)
hidden_concat_layer1 = tf.keras.layers.Dense(32, activation='relu')(dropout_concat_layer)
output_concat_layer = tf.keras.layers.Dense(1, activation='sigmoid')(hidden_concat_layer1)

model = tf.keras.models.Model([lstm_input, state_input], output_concat_layer)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')

In [10]:
model.fit([sequence_train, state_train], label_train, epochs=200, batch_size=128,
          validation_data=([sequence_test, state_test], label_test), verbose=1)
# , callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100)])

Epoch 1/200
9/9 [==============================] - 14s 1s/step - loss: 0.6831 - accuracy: 0.5871 - val_loss: 0.6783 - val_accuracy: 0.6194
Epoch 2/200
9/9 [==============================] - 8s 935ms/step - loss: 0.6868 - accuracy: 0.5696 - val_loss: 0.6617 - val_accuracy: 0.6194
Epoch 3/200
9/9 [==============================] - 8s 930ms/step - loss: 0.6784 - accuracy: 0.5926 - val_loss: 0.6665 - val_accuracy: 0.6194
Epoch 4/200
9/9 [==============================] - 9s 1s/step - loss: 0.6745 - accuracy: 0.5889 - val_loss: 0.6597 - val_accuracy: 0.6194
Epoch 5/200
9/9 [==============================] - 8s 895ms/step - loss: 0.6767 - accuracy: 0.5871 - val_loss: 0.6606 - val_accuracy: 0.6194
Epoch 6/200
9/9 [==============================] - 8s 912ms/step - loss: 0.6776 - accuracy: 0.5908 - val_loss: 0.6577 - val_accuracy: 0.6194
Epoch 7/200
9/9 [==============================] - 8s 905ms/step - loss: 0.6629 - accuracy: 0.5917 - val_loss: 0.6252 - val_accuracy: 0.6323
Epoch 8/200
9/9 [=

In [11]:
djjakfdlksjf

NameError: name 'djjakfdlksjf' is not defined

In [12]:
model.save('./model/checker.h5')